In [6]:
!which pip
!which python
# !pip install numpy torch scikit-learn tqdm
# !pip install pandas ultralytics matplotlib torchvision IPython opencv-python opencv-contrib-python

/workspace/venv/bin/pip
/workspace/venv/bin/python


In [7]:
import os
import numpy as np
import gc
import torch.distributed as dist
import pandas as pd
import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP
import matplotlib.pyplot as plt
from typing import List
import cv2
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms
import pandas as pd
import json
from PIL.Image import Image
import PIL
from tqdm import tqdm
from PIL import ImageDraw
from ultralytics import YOLO
import subprocess

In [8]:
os.chdir("/workspace")

## Infer modern

In [9]:
model = YOLO('yolo_modern_weights.pt')
model("./test/modern/images",save=True, conf=0.25, imgsz=1200, iou=0.5, save_txt=True)


WARNING ⚠️ imgsz=[1200] must be multiple of max stride 32, updating to [1216]
image 1/200 /workspace/test/modern/images/test_1016805_0080.jpg: 1216x1216 1 1_overall, 5 6_headlines, 36 8_textlines, 75.0ms
image 2/200 /workspace/test/modern/images/test_1019174_0012.jpg: 1216x1216 1 1_overall, 1 4_illustration, 12 8_textlines, 74.9ms
image 3/200 /workspace/test/modern/images/test_1020077_0078.jpg: 1216x1216 1 1_overall, 30 8_textlines, 73.9ms
image 4/200 /workspace/test/modern/images/test_1023230_0031.jpg: 1216x1216 1 1_overall, 36 8_textlines, 73.1ms
image 5/200 /workspace/test/modern/images/test_1025409_0065.jpg: 1216x1216 1 1_overall, 1 4_illustration, 2 6_headlines, 54 8_textlines, 73.2ms
image 6/200 /workspace/test/modern/images/test_1027320_0011.jpg: 1216x1216 1 1_overall, 2 4_illustrations, 2 6_headlines, 2 7_captions, 28 8_textlines, 73.0ms
image 7/200 /workspace/test/modern/images/test_1027421_0076.jpg: 1216x1216 1 1_overall, 28 8_textlines, 73.7ms
image 8/200 /workspace/test/mo

[ultralytics.yolo.engine.results.Results object with attributes:
 
 _keys: ('boxes', 'masks', 'probs')
 boxes: ultralytics.yolo.engine.results.Boxes object
 keys: ['boxes']
 masks: None
 names: {0: '0_background', 1: '1_overall', 2: '2_handwritten', 3: '3_typography', 4: '4_illustration', 5: '5_stamp', 6: '6_headline', 7: '7_caption', 8: '8_textline', 9: '9_table'}
 orig_img: array([[[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        ...,
 
        [[  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         .

## Infer old

In [10]:
model = YOLO('yolo_old_weights.pt')
model("./test/old/images",save=True, conf=0.3, imgsz=1200, iou=0.5, save_txt=True)


WARNING ⚠️ imgsz=[1200] must be multiple of max stride 32, updating to [1216]
image 1/211 /workspace/test/old/images/test_10303536_0001.jpg: 1216x1216 1 1_overall, 1 2_handwritten, 2 3_typographys, 2 5_stamps, 74.3ms
image 2/211 /workspace/test/old/images/test_10303703_0031.jpg: 1216x1216 1 1_overall, 20 3_typographys, 73.9ms
image 3/211 /workspace/test/old/images/test_10303741_0022.jpg: 1216x1216 1 1_overall, 36 2_handwrittens, 12 3_typographys, 1 4_illustration, 75.4ms
image 4/211 /workspace/test/old/images/test_10303784_0035.jpg: 1216x1216 1 1_overall, 22 2_handwrittens, 5 3_typographys, 74.8ms
image 5/211 /workspace/test/old/images/test_1100587_0003.jpg: 1216x1216 1 3_typography, 1 4_illustration, 73.4ms
image 6/211 /workspace/test/old/images/test_11038708_0010.jpg: 1216x1216 1 1_overall, 15 3_typographys, 2 4_illustrations, 4 5_stamps, 75.2ms
image 7/211 /workspace/test/old/images/test_11038723_0008.jpg: 1216x1216 1 1_overall, 1 2_handwritten, 33 3_typographys, 73.8ms
image 8/211

[ultralytics.yolo.engine.results.Results object with attributes:
 
 _keys: ('boxes', 'masks', 'probs')
 boxes: ultralytics.yolo.engine.results.Boxes object
 keys: ['boxes']
 masks: None
 names: {0: '0_background', 1: '1_overall', 2: '2_handwritten', 3: '3_typography', 4: '4_illustration', 5: '5_stamp', 6: '6_headline', 7: '7_caption', 8: '8_textline', 9: '9_table'}
 orig_img: array([[[187, 192, 191],
         [188, 193, 192],
         [188, 193, 192],
         ...,
         [195, 200, 199],
         [197, 202, 201],
         [198, 203, 202]],
 
        [[189, 194, 193],
         [189, 194, 193],
         [189, 194, 193],
         ...,
         [195, 200, 199],
         [196, 201, 200],
         [196, 201, 200]],
 
        [[190, 195, 194],
         [190, 195, 194],
         [190, 195, 194],
         ...,
         [194, 199, 198],
         [195, 200, 199],
         [195, 200, 199]],
 
        ...,
 
        [[  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         .